# Discogs Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from albums import albums
from album import album, albumURLInfo
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-07 14:56:14.516724


# Discogs Base Class

In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()
col = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Create Master Pandas DB

In [ ]:
colArtistIDtoName = disc.getCollectionIDToNameData()
colArtistIDtoRefs = disc.getCollectionIDToRefData()
colArtistReftoCnts = disc.getCollectionRefCountsData()

artistIDtoName  = disc.getArtistIDToNameData()
artistIDtoRefs  = disc.getArtistIDToRefData()
albumIDtoName   = disc.getAlbumIDToNameData()
albumIDtoRef    = disc.getAlbumIDToRefData()

artistIDtoAlbumNames = disc.getArtistIDAlbumNames()
artistIDtoAlbumIDs = disc.getArtistIDAlbumIDs()

artistMetaData = disc.getAlbumArtistMetaData()



def splitMetaData(x):
    retval = {}
    if isinstance(x, dict):
        for k,v in x.items():
            retval[k] = [z[0] for z in v.most_common(3)]
    else:
        retval = None
    return retval

from pandas import Series, DataFrame
sArtistToRef  = Series(artistIDtoRefs)
sArtistToName = Series(artistIDtoName)
sAlbumToRef   = Series(albumIDtoRef)
sAlbumToName  = Series(albumIDtoName)

sArtistToAlbums = Series(artistIDtoAlbumIDs)
sArtistToAlbumNames = Series(artistIDtoAlbumNames)
sArtistAlbums = Series([dict(zip(x, y)) for x,y in list(zip(sArtistToAlbums.values, sArtistToAlbumNames))], index=sArtistToAlbums.index)

sArtistMetaData = Series(artistMetaData)
sArtistMetaData = sArtistMetaData.apply(splitMetaData)


cols = ["Ref"]
discdf = DataFrame(sArtistToRef)
discdf.columns = cols
discdf = discdf.join(DataFrame(sArtistToName))
cols += ["Name"]
discdf.columns = cols
tmp = DataFrame(DataFrame(sArtistMetaData)[0].tolist())
tmp.index = sArtistMetaData.index
discdf = discdf.join(tmp)
cols += ["Extra Artists", "Genres", "Styles"]
discdf.columns = cols
discdf = discdf.join(DataFrame(sArtistAlbums))
cols += ["Albums"]
discdf.columns = cols




sColArtistToRef  = Series(colArtistIDtoRefs)
sColArtistToName = Series(colArtistIDtoName)
sColArtistRefToCnts = Series(colArtistReftoCnts)

cols = ["Ref"]
coldiscdf = DataFrame(sColArtistToRef)
coldiscdf.columns = cols
coldiscdf = coldiscdf.join(DataFrame(sColArtistToName))
cols += ["Name"]
coldiscdf.columns = cols

colrefdf = DataFrame(sColArtistRefToCnts)
colrefdf.columns = ["Counts"]
colrefdf.reset_index(inplace=True)
colrefdf.columns = ["Ref", "Counts"]
coldiscdf = coldiscdf.merge(colrefdf, on="Ref")
coldiscdf.index = DataFrame(sColArtistToRef).index

coldiscdf = coldiscdf.merge(discdf, on=["Ref", "Name"], how='left')
coldiscdf.index = DataFrame(sColArtistToRef).index

In [ ]:
artistIDs = disc.getArtistIDToNameData()
db = disc.getAlbumsDBModValData(2)

In [ ]:
from time import sleep
for item in toget.most_common():
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
        sleep(2)
    #print(item)
#len(togetmap)

# Artist Section

In [ ]:
from collections import Counter
db = Counter(disc.getCollectionRefCountsData())

In [ ]:
db18 = disc.getArtistsDBModValData(18)

In [ ]:
for k,v in db18['27518'].mediaCounts.counts.items():
    print(k,len(v))

In [ ]:
for k,v in db18['27518'].media.media.items():
    print(k,len(v))

In [ ]:
ifile="/Volumes/Music/Discog/artists/18/27518.p"
artData = art.getData(ifile)

In [ ]:
artData.mediaCounts.get()

In [ ]:
dbname = "/Volumes/Music/Discog/artists-db/18-DB.p"
dbData = getFile(dbname)

In [ ]:
for k,v in dbData['27518'].mediaCounts.counts.items():
    print(k,'\t',v["All"])

In [ ]:
for k,v in dbData['27518'].media.media.items():
    print(k,'\t',len(v))

In [ ]:
from glob import glob
files = glob("/Volumes/Music/Discog/artists/18/extra/*")

dbData = getFile('/Volumes/Music/Discog/artists-db/18-DB.p')

In [ ]:
media = dbdata['27518'].media.media

In [ ]:
mediaCounts = dbData['27518'].mediaCounts.counts['Releases']

In [ ]:
mediaCounts

In [ ]:
for k,v in media.items():
    retval = {v2.code: True for v2 in v}
    print(k,'\t',mediaCounts.get(k),'\t',len(retval))


In [ ]:
try:
    mediaCounts = dbData['27518'].mediaCounts.counts['Releases']
except:
    mediaCounts = {}

force=False
artistInfo = artist()
for j,ifile in enumerate(files):
    if force is True:
        if j % 250 == 0:
            print("\tProcessed {0}/{1} files.".format(j,len(files)))

        
    

    info     = artistInfo.getData(ifile)
    artistID = info.ID.ID
    
    
    Tretval = {k: {v2.code: True for v2 in v} for k,v in dbData[artistID].media.media.items()}
    TmCnts  = sum([len(v) for k,v in retval.items()])
    
    
    retval = {k: {v2.code: True for v2 in v} for k,v in info.media.media.items()}
    mCnts  = sum([len(v) for k,v in retval.items()])
    
    print("ArtistID: {0}\t{1}\t{2}\t{3}".format(artistID, mediaCounts['All'], TmCnts, mCnts))
    
    keys = list(set(list(info.media.media.keys()) + list(dbData[artistID].media.media.keys())))
    for k in keys:
        v = info.media.media.get(k)
        if v is None:
            continue
        iVal  = {v2.code: v2 for v2 in v}
        dVal  = dbData[artistID].media.media.get(k)
        if dVal is None:
            Tretval = iVal
        else:
            Tretval = {v2.code: v2 for v2 in dVal}
            Tretval.update(iVal)
        dbData[artistID].media.media[k] = Tretval.values()
        print(k,'\t',len(dbData[artistID].media.media[k]),'\t',len(iVal))
    print("")
    
    
for k,v in dbData[artistID].media.media.items():
    print(k,'\t',len(v))
    
if False:
    media = info.media.media
    for mediaType,mediaData in media.items():
        for mediaValues in mediaData:
            code = mediaValues.code
            
        #print("Media: {0} \t{1}".format(mediaType, len(codes)), end='\t')

        if dbdata[artistID].media.media.get(mediaType) is None:
            dbdata[artistID].media.media[mediaType] = mediaData
        else:
            for mediaValues in mediaData:
                if mediaValues.code not in codes:
                    dbdata[artistID].media.media[mediaType].append(mediaValues)

        codes = [mediaValues.code for mediaValues in dbdata[artistID].media.media[mediaType]]

    print("ArtistID: {0}\t{1}".format(artistID, sum([len(x) for x in dbdata[artistID].media.media.values()])))


In [ ]:
sum([len(x) for x in dbData[artistID].media.media.values()])

In [ ]:
ref='/artist/27518-Elvis-Presley'
artistID = dutils.getArtistID(ref)
arts.rmIDFromDB(artistID)
url = arts.getArtistURL(ref)
savename = arts.getArtistSavename(artistID)
arts.downloadArtistURL(url=url, savename=savename, debug=True)
arts.parseArtistModValFiles(18, force=False)

In [ ]:
arts.parseArtistModValExtraFiles(18)

In [ ]:
arts.assertDBModValExtraData(modVal=18)

In [ ]:
#for modval in range(18,19):

In [ ]:
def findErrs(modVal, show=True):
    print("Finding Errors For {0}".format(modVal))
    dbdata = disc.getArtistsDBModValData(modVal)

    dels = []
    for artistID,artistInfo in dbdata.items():
        ref    = artistInfo.url.url

        nMedia = sum([len(x) for x in artistInfo.media.media.values()])
        counts = 0 #artistInfo.mediaCounts.counts["Releases"]["All"]
        pages  = artistInfo.pages
        if pages.tot is None:
            print(artistID,'\t',ref,'\t',nMedia,'\t',pages.get(),'\t',counts)
            if show:
                continue
                
            dels.append(artistID)
        elif (pages.tot > pages.ppp or nMedia % 25 == 0) and pages.pages > 1:
            print(artistID,'\t',ref,'\t',nMedia,'\t',pages.get(),'\t',counts)
            if show:
                continue
                
            dels.append(artistID)
            
            artistID = dutils.getArtistID(ref)
            url = arts.getArtistURL(ref)
            savename = arts.getArtistSavename(artistID)
            arts.downloadArtistURL(url=url, savename=savename, debug=True, force=True)

    #arts.rmIDFromDB(dels)
    if show:
        return

    saveVal = False
    for ID in dels:
        try:
            del dbdata[ID]
            print("Deleted {0}".format(ID))
            saveVal = True
        except:
            print("Not there...")
            
    dbname = disc.getArtistsDBModValFilename(modVal)
    if saveVal:
        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)
    else:
        print("No reason to save {0}".format(dbname))

    arts.parseArtistModValFiles(modVal, force=False)
    arts.assertDBModValExtraData(modVal)
    arts.parseArtistModValExtraFiles(modVal)

In [ ]:
# 92
for modVal in range(93,100):
    findErrs(modVal, show=False)

In [ ]:
modVal=None
if modVal is not None:
    arts.assertDBModValExtraData(modVal)
    arts.parseArtistModValExtraFiles(modVal)

# Download Artist Albums In Order

In [ ]:
albs.downloadAlbumsFromArtists(maxAlbums=1)

Loading /Volumes/Music/Discog/artists-db/0-DB.p... Found 4520 Artists
1 / 4520 	: 1000 	 Dave Clarke
	5 entries of type Albums
2 / 4520 	: 1000500 	 Club Pulse
3 / 4520 	: 100200 	 Dike
	1 entries of type Albums
4 / 4520 	: 1002000 	 Larry Stokes
5 / 4520 	: 1002600 	 Gidd Sanchez
6 / 4520 	: 1002900 	 John Keding
7 / 4520 	: 1003900 	 Jason Pancho
8 / 4520 	: 1005400 	 Gapeseed
	2 entries of type Albums
9 / 4520 	: 1007400 	 The Beauty Shop
	2 entries of type Albums
10 / 4520 	: 1010100 	 The Salads
	4 entries of type Albums
11 / 4520 	: 1013700 	 Mother Mother
	5 entries of type Albums
12 / 4520 	: 1014400 	 Lucious Randolph
13 / 4520 	: 1016300 	 Poul Ruders
	19 entries of type Albums
14 / 4520 	: 101700 	 Lanu
	2 entries of type Albums
15 / 4520 	: 102100 	 Extrabreit
	19 entries of type Albums
  Found album code 473498 with 1 artists
  Saving /Volumes/Music/Discog/albums/0/102100/473498.p
  --> This file is 17.6kB.
  Done and sleeping...
16 / 4520 	: 1027500 	 Luigi Barion
17 / 45

87 / 4520 	: 112500 	 Xque?
	1 entries of type Albums
88 / 4520 	: 1127900 	 Andy MacQueen
89 / 4520 	: 112800 	 Exile (4)
	7 entries of type Albums
90 / 4520 	: 1130300 	 The Hats
91 / 4520 	: 1130400 	 The Good Old Boys
	2 entries of type Albums
92 / 4520 	: 1130500 	 Ghettolandz
	1 entries of type Albums
93 / 4520 	: 1132500 	 Joke Schonewille
94 / 4520 	: 1132700 	 The Black Hollies
	4 entries of type Albums
95 / 4520 	: 113300 	 George B.
96 / 4520 	: 1136200 	 Rob Wunderman
97 / 4520 	: 113700 	 Steve Maestro
98 / 4520 	: 1137900 	 You Can't Have A Dinosaur
99 / 4520 	: 1138200 	 Tritonal
	5 entries of type Albums
100 / 4520 	: 1139200 	 Yosi Horikawa
	1 entries of type Albums
101 / 4520 	: 1140900 	 Fred Waring And The Glee Club
	1 entries of type Albums
102 / 4520 	: 1141300 	 Wendo Kolosoy
	6 entries of type Albums
103 / 4520 	: 1144000 	 Gasoline Queen
104 / 4520 	: 1147100 	 Renato Cellini
	9 entries of type Albums
  Found album code 8962750 with 5 artists
  Saving /Volumes/

  Found album code 6419781 with 1 artists
  Exists /Volumes/Music/Discog/albums/90/2574490/6419781.p
  Done and sleeping...
169 / 4520 	: 1244200 	 S. & M. Young
170 / 4520 	: 1246900 	 Baby (7)
	2 entries of type Albums
171 / 4520 	: 125100 	 The Quireboys
	20 entries of type Albums
  Found album code 608830 with 1 artists
  Saving /Volumes/Music/Discog/albums/0/125100/608830.p
  --> This file is 16.9kB.
  Done and sleeping...
172 / 4520 	: 1251900 	 Mika Ueki
173 / 4520 	: 125500 	 Unknown (69)
	1 entries of type Albums
174 / 4520 	: 1257800 	 ãã«ãã¿ã¤ã-08
	1 entries of type Albums
  Found album code 1507638 with 1 artists
  Exists /Volumes/Music/Discog/albums/NAN/NAN/1507638.p
  Done and sleeping...
175 / 4520 	: 1258500 	 Royce (3)
	2 entries of type Albums
176 / 4520 	: 1258800 	 Luisa De Santis
177 / 4520 	: 1259000 	 Imbibing Bile
178 / 4520 	: 12600 	 Sequel
	1 entries of type Albums
179 / 4520 	: 1263100 	 Program 81
	3 entries of type Albums
180 / 4520 	: 1264600 	 N

  Found album code 5507551 with 2 artists
  Exists /Volumes/Music/Discog/albums/92/259092/5507551.p
  Saving /Volumes/Music/Discog/albums/0/1378200/5507551.p
  --> This file is 15.1kB.
  Done and sleeping...
256 / 4520 	: 1378300 	 Parker Chandler
257 / 4520 	: 1378400 	 Jon Fisher (2)
258 / 4520 	: 1378600 	 Denzal Park
259 / 4520 	: 1378700 	 Essents
260 / 4520 	: 1379000 	 Donie Cassidy
261 / 4520 	: 138000 	 Mathieu Werchowski
	6 entries of type Albums
262 / 4520 	: 1380200 	 Valerie Kägi
	1 entries of type Albums
263 / 4520 	: 138100 	 Mercy
	2 entries of type Albums
264 / 4520 	: 1381600 	 Magda Mayas
	13 entries of type Albums
  Found album code 4512985 with 2 artists
  Saving /Volumes/Music/Discog/albums/47/280447/4512985.p
  --> This file is 17.2kB.
  Saving /Volumes/Music/Discog/albums/0/1381600/4512985.p
  --> This file is 17.2kB.
  Done and sleeping...
265 / 4520 	: 1381900 	 Heiner Merk
	1 entries of type Albums
266 / 4520 	: 1383000 	 Lasse Hoikka
	6 entries of type Album

355 / 4520 	: 1481200 	 Jackson Lee
	1 entries of type Albums
356 / 4520 	: 1482900 	 Russ (15)
	2 entries of type Albums
357 / 4520 	: 148400 	 B.A.D.
358 / 4520 	: 1485000 	 Milan Princ
	3 entries of type Albums
359 / 4520 	: 148600 	 Knowledge Of Bugs
	2 entries of type Albums
360 / 4520 	: 1486500 	 Theophilus London
	7 entries of type Albums
361 / 4520 	: 1486700 	 STR8 Funk
362 / 4520 	: 1488100 	 Robert Passow
363 / 4520 	: 1489800 	 Shirley Zwerus' Choir
364 / 4520 	: 1489900 	 Cold Embrace Of Mythical Infinite Season
365 / 4520 	: 1493300 	 David Chan
	1 entries of type Albums
366 / 4520 	: 1494400 	 Sats
367 / 4520 	: 1496500 	 Fernando Montesinos
368 / 4520 	: 1497500 	 Francesco Gatta
369 / 4520 	: 1500 	 Paranoid Jack
	1 entries of type Albums
370 / 4520 	: 15000 	 A Sense Of Summer
371 / 4520 	: 1500000 	 Torino Reitano
	1 entries of type Albums
372 / 4520 	: 1501700 	 D Fresh
373 / 4520 	: 1504800 	 Jazz Fiddlers
	3 entries of type Albums
374 / 4520 	: 1506200 	 Punishme

...

  Found album code 13590709 with 1 artists
  Exists /Volumes/Music/Discog/albums/1/7142201/13590709.p
  Done and sleeping...
907 / 4520 	: 2115800 	 Carol Vaness
	13 entries of type Albums
  Found album code 8781473 with 6 artists
  Saving /Volumes/Music/Discog/albums/53/369053/8781473.p
  --> This file is 20.1kB.
  Saving /Volumes/Music/Discog/albums/0/2115800/8781473.p
  --> This file is 20.1kB.
  Saving /Volumes/Music/Discog/albums/25/3420625/8781473.p
  --> This file is 20.1kB.
  Exists /Volumes/Music/Discog/albums/99/1710599/8781473.p
  Saving /Volumes/Music/Discog/albums/19/27519/8781473.p
  --> This file is 20.1kB.
  Saving /Volumes/Music/Discog/albums/32/842132/8781473.p
  --> This file is 20.1kB.
  Done and sleeping...
908 / 4520 	: 2116000 	 Malte Preuß
909 / 4520 	: 2117700 	 Peter Uherčík
	1 entries of type Albums
910 / 4520 	: 2119100 	 Hazelpark
911 / 4520 	: 212000 	 Elisabeth Chojnacka
	19 entries of type Albums
912 / 4520 	: 2120900 	 Joe Quinones
913 / 4520 	: 2121500

  Found album code 197680 with 3 artists
  Exists /Volumes/Music/Discog/albums/31/369031/197680.p
  Saving /Volumes/Music/Discog/albums/0/222600/197680.p
  --> This file is 19.8kB.
  Exists /Volumes/Music/Discog/albums/46/254546/197680.p
  Done and sleeping...
1012 / 4520 	: 2227500 	 Fred Gito
1013 / 4520 	: 2227700 	 Holy Filament
	2 entries of type Albums
1014 / 4520 	: 222800 	 Chilla Frauste
	1 entries of type Albums
1015 / 4520 	: 2228200 	 Ray Lugo & The Boogaloo Destroyers
	2 entries of type Albums
1016 / 4520 	: 223100 	 Kelvin Wooten
1017 / 4520 	: 223200 	 Triosk
	3 entries of type Albums
1018 / 4520 	: 2232100 	 Josph G. Bilby
1019 / 4520 	: 2233800 	 Oliva Morga
1020 / 4520 	: 2235200 	 Gwen Sao
1021 / 4520 	: 2235700 	 C. Taylor Crothers
1022 / 4520 	: 2236900 	 Kaibe Takeshi
1023 / 4520 	: 2237600 	 Embassy Of Silence
	3 entries of type Albums
1024 / 4520 	: 2238600 	 Edmund Teske
1025 / 4520 	: 2241500 	 Ronald Ferdinand
1026 / 4520 	: 2242700 	 Saul Lambert
1027 / 4520

1107 / 4520 	: 2334400 	 The Tokyo Cuban Boys
	42 entries of type Albums
  Found album code 13666606 with 1 artists
  Saving /Volumes/Music/Discog/albums/0/2334400/13666606.p
  --> This file is 15.3kB.
  Done and sleeping...
1108 / 4520 	: 2334900 	 Robert Wheeler (2)
1109 / 4520 	: 233700 	 Mike Shiflet
	44 entries of type Albums
  Found album code 536913 with 1 artists
  Saving /Volumes/Music/Discog/albums/0/233700/536913.p
  --> This file is 14.8kB.
  Done and sleeping...
1110 / 4520 	: 2338000 	 M. S.
1111 / 4520 	: 2339900 	 Bronco Army
	4 entries of type Albums
1112 / 4520 	: 23400 	 Infinite Mass
	7 entries of type Albums
  Found album code 872183 with 1 artists
  Saving /Volumes/Music/Discog/albums/0/23400/872183.p
  --> This file is 15.5kB.
  Done and sleeping...
1113 / 4520 	: 234200 	 Pete Simpson (2)
	1 entries of type Albums
1114 / 4520 	: 2342800 	 Sick City
	1 entries of type Albums
1115 / 4520 	: 234300 	 Discipline Unit
1116 / 4520 	: 2343200 	 John Dee (6)
1117 / 4520

  Found album code 5648458 with 1 artists
  Exists /Volumes/Music/Discog/albums/NAN/NAN/5648458.p
  Done and sleeping...
1197 / 4520 	: 246300 	 Tonic (3)
1198 / 4520 	: 2463700 	 Ben (142)
1199 / 4520 	: 2467600 	 Guy Gappu
1200 / 4520 	: 2468200 	 Paradiso (9)
1201 / 4520 	: 246900 	 Drumatik
	4 entries of type Albums
1202 / 4520 	: 247000 	 Mark Opitz
1203 / 4520 	: 2470400 	 Grindcrusher
1204 / 4520 	: 247100 	 Giorge Pettus
	2 entries of type Albums
1205 / 4520 	: 2471400 	 Ansambl "Stari Zagreb"
1206 / 4520 	: 2474500 	 Krieg (7)
1207 / 4520 	: 2474800 	 Illinformed
	3 entries of type Albums
1208 / 4520 	: 2477300 	 Monica Bacelli
	2 entries of type Albums
1209 / 4520 	: 2477600 	 Old Fashioned Oyster Crackers
1210 / 4520 	: 2479000 	 Miroslav Grčev
1211 / 4520 	: 2480800 	 Andrew Cross (3)
1212 / 4520 	: 2482200 	 Toast (13)
1213 / 4520 	: 2483900 	 Raw Yak Meat
	2 entries of type Albums
1214 / 4520 	: 2484500 	 Hein Braat
	5 entries of type Albums
1215 / 4520 	: 2485300 	 Anssi

  --> This file is 16.3kB.
  Saving /Volumes/Music/Discog/albums/60/3442060/8088775.p
  --> This file is 16.3kB.
  Done and sleeping...
1292 / 4520 	: 2558700 	 Rimmersgard
	2 entries of type Albums
1293 / 4520 	: 2558900 	 Maximilian Schriever
1294 / 4520 	: 2560900 	 I Menestrelli Della Canzone
	2 entries of type Albums
1295 / 4520 	: 256100 	 J. J. Barnes
	3 entries of type Albums
1296 / 4520 	: 2562300 	 Tanikawa Megumi
1297 / 4520 	: 2562700 	 Sashi
1298 / 4520 	: 2562900 	 Andrea Lindsay
	4 entries of type Albums
  Found album code 11539665 with 1 artists
  Saving /Volumes/Music/Discog/albums/0/2562900/11539665.p
  --> This file is 15.4kB.
  Done and sleeping...
1299 / 4520 	: 256500 	 Michael Harris
	2 entries of type Albums
1300 / 4520 	: 2565400 	 Joe Keady
1301 / 4520 	: 2568500 	 Grandsinister Ice
1302 / 4520 	: 256900 	 DJ Rondevu
	5 entries of type Albums
  Found album code 10706476 with 1 artists
  Saving /Volumes/Music/Discog/albums/0/256900/10706476.p
  --> This file is

# Download Most Common Artist Albums

In [ ]:
from collections import Counter
db = Counter(disc.getCollectionRefCountsData())
coreAlbumsRefs = disc.getArtistIDCoreAlbumRefs()
artistMetaData = disc.getAlbumArtistMetaData()
albumDB = disc.getAlbumIDToRefData()

In [ ]:
toget = {}
for item in db.most_common(5000):
    ref,cnt  = item[0],item[1]
    artistID = dutils.getArtistID(ref)
    modVal   = str(int(artistID[-2:]))
    
    genre = artistMetaData[artistID]['Genre'].most_common(1)[0][0]
    #if genre not in ["Rock", "Pop", "Hip Hop"]:
    #    continue

    if toget.get(artistID) is None:
        toget[artistID] = {}
        
    
    coreAlbums = coreAlbumsRefs.get(artistID)
    #if len(coreAlbums) % 25 == 0:
    #print("{0: <50}{1: <30}{2: <10}{3}".format(ref,genre,artistID,len(coreAlbums)))
        
    for albumID,albumRef in coreAlbums.items():
        haveIt = albumDB.get(albumID)
        #print("\t{0: <15}{1}".format(albumID,haveIt))
        if haveIt is None:
            toget[artistID][albumID] = albumRef
            
    print("{0: <15}{1: <30}{2: <5}{3: <5}".format(artistID, genre, len(toget[artistID]), len(coreAlbums)))
        
    continue
    if False:
        print(artistID,'\t',ref)
        arts.rmIDFromDB(artistID)
        url = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        arts.downloadArtistURL(url=url, savename=savename)
        sleep(2)
    #print(artistID,item,len(coreAlbums))

In [ ]:
len(toget)

In [ ]:
known = {}

In [ ]:
# Ignore 0-311 (start with Carl Perkins)
known = getFile("ignoreAlbum.p")
for ia,(artistID,albumRefs) in enumerate(toget.items()):
    for i,(albumID,albumRef) in enumerate(albumRefs.items()):
        url = albs.getAlbumURL(albumRef)
            
        savename = albs.getAlbumSavename(artistID, albumID)
        if isFile(savename):
            continue
            
        if known.get(url) is True:
            continue

        if False:
            known[url] = True
            continue
            
        print(artistID,'\t',ia,'\t',len(toget),'\t\t',i,'/',len(albumRefs),'\t',artistID)
        print("\tGetting: {0}".format(savename))
        retval = albs.downloadAlbumURLData(url=url, savename=savename, artistID=artistID, sleeptime=1.75)
        if retval is False:
            known[url] = True
            if len(known) % 5 == 0:
                saveFile(idata=known, ifile="ignoreAlbum.p")

In [ ]:
len(known)

In [ ]:
saveFile(idata=known, ifile="ignoreAlbum.p")

In [ ]:
known = getFile("ignoreAlbum.p")

In [ ]:
for artistID,artistRefs in toget.items():
    print(artistID,len(artistRefs))

In [ ]:
tmp = disc.getAlbumIDToArtistIDData()

In [ ]:
db30 = disc.getAlbumsDBModValData(44)
from collections import Counter

In [ ]:
artistMetaData = {}
for artistID,artistData in db30.items():
    print(artistID)
    artistMetaData[artistID] = {"Genre": Counter(), "Artists": Counter(), "Style": Counter()}
    for albumID,albumData in artistData.items():
        genres = albumData.profile.genre
        if not isinstance(genres, list):
            genres = [genres]
        for genre in genres:
            artistMetaData[artistID]['Genre'][genre.name] += 1
        
        artists = albumData.artist.artists
        for artist in artists:
            artistMetaData[artistID]['Artists'][artist.name] += 1
        
        styles = albumData.profile.style
        if not isinstance(styles, list):
            styles = [styles]
        for style in styles:
            artistMetaData[artistID]['Style'][style.name] += 1

In [ ]:
artistMetaData

In [ ]:
albdata = albs.getData(ifile)

In [ ]:
for k,v in albdata.get().items():
    print(k,'\t',v.get())

In [ ]:
ArtistAlbums = disc.getArtistIDCoreAlbumIDs()
AlbumIDToName = disc.getAlbumIDToNameData()

In [ ]:
from string import ascii_uppercase, digits, punctuation
vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
AsciiToName = {val: [x for x in NameToIDs.keys() if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}

In [ ]:
from discogsBase import discogs
from string import ascii_uppercase, digits, punctuation
from searchUtils import findNearest

class discdb:
    def __init__(self, disc):
        self.dutils    = discogsUtils()
        self.disc      = disc
        
        ## DBs
        self.IDToName  = None
        self.NameToIDs = None
        self.AsciiToName = None
        self.ArtistAlbums = None
        self.AlbumIDToName = None
        
        self.artistRetval = None
        self.minCutoff    = 0.85
        self.minN         = 1
        
        
    def loadAlbumDBs(self, ArtistAlbums=None, AlbumIDToName=None):
        start, cmt = clock("Loading Album DBs")
        if ArtistAlbums is None:
            self.ArtistAlbums = disc.getArtistIDCoreAlbumNames()
        else:
            self.ArtistAlbums = ArtistAlbums
            
        if AlbumIDToName is None:
            self.AlbumIDToName  = disc.getAlbumIDToNameData()
        else:
            self.AlbumIDToName  = AlbumIDToName
        elapsed(start, cmt)
        
        
    def loadArtistDBs(self, IDToName=None, NameToIDs=None, AsciiToName=None):
        start, cmt = clock("Loading Artist DBs")
        
        if IDToName is None:
            self.IDToName  = disc.getArtistIDToNameData()
        else:
            self.IDToName  = IDToName
            
        if NameToIDs is None:
            self.NameToIDs = disc.getArtistNameToIDsData()
        else:
            self.NameToIDs = NameToIDs
            
        if AsciiToName is None:
            vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
            self.AsciiToName = {val: [x for x in n_items if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}
        else:
            self.AsciiToName = AsciiToName
            
        elapsed(start, cmt)

        
    def findArtist(self, name, num=None, cutoff=None):
        print("Searching For {0}".format(name))
        try:
            charval = str(name)[0]
        except:
            raise ValueError("Artist must be a string Type=[{0}]".format(type(name)))
            
        if num is None:
            num = self.minN
            
        if cutoff is None:
            cutoff = self.minCutoff

        chardata = self.AsciiToName.get(charval)
        if chardata is None:
            raise ValueError("There are no entries for character [{0}]".format(charval))
        retval = findNearest(name, chardata, num=num, cutoff=cutoff)
        print("Results: {0}".format(retval))
        self.artistRetval = retval
        
        return retval
    
    
    def findArtistAlbums(self, artist=None):
        if artist is None:
            artist = self.artistRetval
        if artist is None:
            print("Artist is None!")
            return
        
        if isinstance(artist, str):
            artists = [artist]
        else:
            artists = artist
            
        artistIDs = []
        for artist in artists:
            IDs = self.NameToIDs.get(artist)
            if isinstance(IDs,list):
                retval = [ID for ID in IDs if ID is not None]
                artistIDs += retval                
        artistIDs = list(set(artistIDs))

        for artistID in artistIDs:
            artist = self.IDToName[artistID]
            print("Artist [{0}]  ---->  [{1}]".format(artist, artistID))
            albumIDs = ArtistAlbums.get(artistID)
            print("  ID [{0}]  ---->  {1} albums".format(artistID, len(albumIDs)))
            for albumID in albumIDs:
                albumNames = self.AlbumIDToName.get(albumID)
                if isinstance(albumNames, dict):
                    albumNames = list(albumNames.keys())
                print('\t{0: <12}\t{1}'.format(albumID, albumNames))
            
        

In [ ]:
ddb = discdb(disc)
ddb.loadArtistDBs(IDToName=IDToName, NameToIDs=NameToIDs, AsciiToName=AsciiToName)
ddb.loadAlbumDBs(ArtistAlbums=ArtistAlbums, AlbumIDToName=AlbumIDToName)
ddb.findArtist("Prince", num=1)
ddb.findArtistAlbums()

In [ ]:
%load_ext autoreload
%autoreload
disc   = discogs()
arts   = artists(disc)
albs   = albums(disc)
dutils = discogsUtils()
col    = collections(disc)

for modVal in list(range(99, 50, -1)):
    albs.downloadAlbumModValData(modVal, maxAlbums=1)

In [ ]:
IDToName    = disc.getArtistIDToNameData()
IDToRef     = disc.getArtistIDToRefData()
AlbIDToName = disc.getAlbumIDToNameData()
AlbIDToRef  = disc.getAlbumIDToRefData()

IDToCoreAlbID = disc.getArtistIDCoreAlbumIDs()
IDToAlbID     = disc.getArtistIDAlbumIDs()

In [ ]:
df = DataFrame.fromIDToCoreAlbID

In [ ]:
x   = {k: ":".join(v) for k,v in IDToCoreAlbID.items()}
pdf = DataFrame.from_dict(x, orient='index')

In [ ]:
pdf

In [ ]:
from itertools import islice
def takeN(n, iterable):
    if isinstance(iterable, dict):
        "Return first n items of the iterable as a list"
        #keys = takeN(n, iterable.keys())
        keys = list(islice(iterable, n))        
        return {k: iterable[k] for k in keys}
    elif isinstance(iterable, list):
        "Return first n items of the iterable as a list"
        return list(islice(iterable, n))
    else:
        print("No idea about {0}".format(type(iterable)))
        return None


#n_items = takeN(500000, ddb.NameToIDs.keys())
#n_items = ddb.NameToIDs.keys()

# Albums DataFrame

In [ ]:
db = disc.getAlbumsDBModValData(2)

In [ ]:
pData = {}
nAlbs = 0
for i,(artistID,artistData) in enumerate(db.items()):
    nAlbs += len(artistData)
    #print(i,'/',len(db),'\t',nAlbs,'\t',artistID)
    for albumID,albumData in artistData.items():
        rowData = {}
        
        
        ##dict_keys(['artist', 'album', 'profile', 'url', 'code', 'tracks', 'versions', 'credits', 'err'])
        
        
        
        ################################################
        # Code
        ################################################
        code = albumData.code.code
        if code != albumID:
            raise ValueError(code,albumID)
              
                
        ################################################
        # Profile
        ################################################
        profile = albumData.profile
        for key,value in profile.get().items():
            #print(key,value,type(value))
            if value is not None:
                if isinstance(value, list):
                    name = "::".join(x.name for x in value)
                elif isinstance(value, albumURLInfo):
                    name = value.name
                else:
                    print(value)
                    1/0
                
                rowData[key] = name
              
                
        ################################################
        # Credits
        ################################################
        credits = albumData.credits.credit
        names = []
        for key,value in credits.items():
            if isinstance(value, list):
                names = value
            elif isinstance(value, albumURLInfo):
                names.append(value.name)
            else:
                print(value,type(value))
                1/0

        key='credits'
        rowData[key] = "::".join(x.name for x in names)
                
        
        
        ################################################
        # Album Versions
        ################################################
        versions = albumData.versions
        rowData['versions'] = versions.nums
        
        
        ################################################
        # Album Name
        ################################################
        key = 'album'
        albumName = albumData.album.name
        rowData[key] = albumName
        
        
        ################################################
        # Album URL
        ################################################
        key = 'ref'
        albumName = albumData.url.url
        rowData[key] = albumName
        #print(key,'\t',album)
        
        
        ################################################
        # Artists
        ################################################
        artistNames = []
        artistIDs   = []

        artists = albumData.artist.artists
        for artist in artists:
            ID = artist.ID            
            if ID is not None:
                artistNames.append(artist.name)
                artistIDs.append(artist.ID)
            #print(artist[0].get())
        #print(artistNames,'\t',artistIDs)
        rowData["artist"]   = "::".join(artistNames)
        rowData["artistID"] = "::".join(artistIDs)
                
        pData[albumID] = rowData
        #print('\t\t',rowData.keys())

In [ ]:
from pandas import DataFrame
df = DataFrame.from_dict(pData, orient='index')

def toDate(x):
    retval = None
    if retval is None:
        try:
            retval = to_datetime(x, format='%d %b %Y')
        except:
            retval = None
        
    if retval is None:
        try:
            retval = to_datetime(x, format='%b %Y')
        except:
            retval = None
            
    if retval is None:
        try:
            retval = to_datetime(x, format='%Y')
        except:
            retval = None

    return retval

df['released'] = df['released'].apply(toDate)
for col in ['label', 'aformat', 'country', 'genre', 'style', 'credits', 'artist', 'artistID']:
    df[col] = df[col].str.split("::")

In [ ]:
df.head()

In [ ]:
df.versions.unique()

In [ ]:
saveFile(idata=df, ifile="test.p")

In [ ]:
df.

In [ ]:
x = '27 Oct 2016'

In [ ]:
from pandas import to_datetime

In [ ]:
to_datetime(x, format='%d %b %Y')

In [ ]:
from pandas import DataFrame
dfAlb = DataFrame(tmp.values(), index=tmp.keys())
dfAlb.columns = ["Albums"]

In [ ]:
#takeN(4, IDToCoreAlbID)

In [ ]:
from pandas import DataFrame
dfAlbName = DataFrame(AlbIDToName.values(), index=AlbIDToName.keys())
dfAlbName.columns = ["Album"]

dfAlbRef = DataFrame(AlbIDToRef.values(), index=AlbIDToRef.keys())
dfAlbRef.columns = ["AlbumRef"]


dfAlb = dfAlbName.join(dfAlbRef)

In [ ]:
dfAlb.head()

In [ ]:
from pandas import DataFrame
dfName = DataFrame(IDToName.values(), index=IDToName.keys())
dfName.columns = ["Artist"]

dfRef = DataFrame(IDToRef.values(), index=IDToRef.keys())
dfRef.columns = ["ArtistRef"]

x   = {k: ":".join(v) for k,v in IDToCoreAlbID.items()}
dfArtCoreAlb = DataFrame.from_dict(x, orient='index')
dfArtCoreAlb.columns = ["ArtistAlbums"]

x   = {k: ":".join(v) for k,v in IDToAlbID.items()}
dfArtAlb = DataFrame.from_dict(x, orient='index')
dfArtAlb.columns = ["ArtistAlbums"]

dfArt = dfName.join(dfRef)
dfArt = dfArt.join(dfArtAlb)

In [ ]:
dfArt.head()

In [ ]:
def getAlbumNames(x):
    #for code in x.split(":"):
    return len(x.split(":"))
    #return dfAlb.loc[dfAlb.index.isin(x.split(":"))].shape
dfArt.head()['ArtistAlbums'].apply(getAlbumNames)

In [ ]:
dfArt.head()['ArtistAlbums']

In [ ]:
db = disc.getAlbumsDBModValData(2)

In [ ]:
getClose(x, )
difflib.get_close_matches(word, possibilities[, n][, cutoff])¶

In [ ]:
import difflib
#artistNames = difflib.get_close_matches("John Mayer", dfArt["Artist"], 3, 0.55)

colData = dfArt["Artist"]
idx     = colData.index

In [ ]:
retval = idx[colData.isin(difflib.get_close_matches("John Mayer ", colData, 3, 0.99))]

In [ ]:
df.loc[retval]

In [ ]:
start,cmt = clock()
for i in range(2):
    dfArt["Artist"][dfArt["Artist"] == 'John Mayer'].index[0]
elapsed(start, cmt)

In [ ]:
start,cmt = clock()
for i in range(2):
    print(dfArt.index[dfArt["Artist"] == 'John Mayer'])
elapsed(start, cmt)

In [ ]:
(dfArt["Artist"] == 'John Mayer')

In [ ]:
dfArt["Artist"].apply(lambda x: x[:5])

In [ ]:
d = {'1000500': ['34311435'],
 '100200': ['2843377',
  '2082992',
  '1561704',
  '50511',
  '3019891',
  '50530',
  '7516271',
  '2732930',
  '219887'],
 '1002000': []}
d

In [ ]:
import numpy as np
my_dict = dict( A = np.array([1,2]), B = np.array([1,2,3,4]) )
df = DataFrame.from_dict(d, orient='index')
#df.transpose()
#DataFrame(d, dtype='object')

In [ ]:
x = {k: ":".join(v) for k,v in d.items()}
DataFrame.from_dict(x, orient='index')

In [ ]:
dfArt.head()

In [ ]:
df

In [ ]:
from itertools import islice
def takeN(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = takeN(500000, ddb.NameToIDs.keys())
n_items = ddb.NameToIDs.keys()

In [ ]:
import math
from collections import Counter

def eta(data, unit='natural'):
    base = {
        'shannon' : 2.,
        'natural' : math.exp(1),
        'hartley' : 10.
    }

    if len(data) <= 1:
        return 0

    counts = Counter()

    for d in data:
        counts[d] += 1

    ent = 0

    probs = [float(c) / len(data) for c in counts.values()]
    for p in probs:
        if p > 0.:
            ent -= p * math.log(p, base[unit])

    return ent

In [ ]:
eta(['mary', 'had', 'a', 'little', 'lamb'])

In [ ]:
eta([c for c in "mary had a little lamb"])

In [ ]:
eta([['mary', 'had'], ['a', 'little', 'lamb']])

In [ ]:
from string import ascii_uppercase, digits, punctuation
vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
vals

In [ ]:
retval = {val: [x for x in n_items if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}

In [ ]:
print(test)
for k,v in retval.items():
    print(k, findNearest(test, v, num=1, cutoff=0.8))

In [ ]:
for k,v in retval.items():
    print(k,len(v))

In [ ]:
from searchUtils import findNearest
test = "Poul 3 RudersX"

In [ ]:
start, cmt = clock("Test")
retval = []
ifn    = {0: 0.95, 1: 0.80}
for i,cutoff in ifn.items():
    retval = findNearest(test, retval[test[0]], num=1, cutoff=cutoff)
    if len(retval) > 0:
        break
elapsed(start, cmt)
print(retval)



start, cmt = clock("Test")
retval = []
ifn    = {0: 0.95}
for i,cutoff in ifn.items():
    retval = findNearest(test, retval[test[0]], num=1, cutoff=cutoff)
    if len(retval) > 0:
        break
elapsed(start, cmt)
print(retval)

In [ ]:
for i,(k,v) in enumerate(ddb.IDToName.items()):
    print(i,k,v)
    if i > 50:
        break

In [ ]:
for i,(k,v) in enumerate(ddb.NameToIDs.items()):
    print(i,k,v)
    if i > 50:
        break

In [ ]:
%load_ext autoreload
%autoreload
from artist import artist
from artists import artists
art  = artist()
arts = artists(disc)
if False:
    arts.parseArtistFiles(force=False)
if False:
    for modVal in range(0, 100):
        arts.parseArtistModValFiles(modVal)#art.searchDiscogForArtist("$uicideboy$")
arts.buildMetadata(force=True)



In [ ]:
dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
refs = [dbCol.get('1930579')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if False:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
arts.parseArtistModValFiles(modVal=89, force=True)

In [ ]:
errs=[]
for modVal in range(51,52):
    print("\n===============================================================")
    print("assertDBModValData({0})".format(modVal))
    if True:
        try:
            arts.assertDBModValData(modVal)
        except:
            errs.append(modVal)
    if False:
        try:
            arts.assertDBModValExtraData(modVal)
        except:
            errs.append(modVal)

In [ ]:
data = art.getData('/Volumes/Music/Discog/artists/22/2047522.p')

In [ ]:
errs

In [ ]:
arts.parseArtistModValFiles(modVal=51, force=True)

In [ ]:
for modVal in range(95, 100):
    print("ModVal = {0}".format(modVal))
    arts.assertDBModValData(modVal)
    
#arts.assertDBModValExtraData(modVal)

In [ ]:
dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
arts.rmIDFromDB('334885', '85')

In [ ]:
vals="""3,818 Vocal
2,446 Rock & Roll
2,014 Easy Listening
1,640 Romantic
1,200 Big Band
1,004 Swing
887 Classical
692 Opera
634 Country
582 Dixieland
560 Ballad
550 Bop
541 Folk
500 Soundtrack
477 Musical
462 Cool Jazz
447 Baroque
443 Rhythm & Blues
433 Modern
393 Rockabilly
363 Pop Rock
307 Doo Wop
274 Novelty
249 Theme
208 Calypso
203 Skiffle
200 Religious
180 Celtic
179 Ragtime
177 Hard Bop
174 Comedy
170 Spoken Word
167 Instrumental
133 Latin Jazz
127 Cha-Cha
127 Mambo
119 Gospel
117 Score
106 Chanson
99 Space-Age
96 Impressionist
94 Music Hall
91 Choral
91 Soul
90 Contemporary Jazz
88 Blues Rock
86 Post Bop
81 Light Music
79 Marches
78 Poetry
66 Operetta
65 Country Rock
64 Piano Blues
59 Neo-Classical
59 Parody
56 Neo-Romantic
55 Rumba
54 African
54 Tango
53 Honky Tonk
52 Afro-Cuban Jazz
52 Country Blues
52 Field Recording
52 Story
51 Soul-Jazz
50 Afro-Cuban
50 Pacific
43 Brass Band
43 Samba
40 Nursery Rhymes
39 Contemporary
39 Military
39 Renaissance
38 Dialogue
38 Flamenco
38 Schlager
33 Pipe & Drum
31 Bolero
25 Canzone Napoletana
25 Classic Rock
25 Radioplay
24 Boogie Woogie
24 Fado
24 Monolog
23 Technical
22 Bluegrass
21 Lounge
21 Mento
21 Oratorio
21 Surf
20 Special Effects
19 Delta Blues
19 Education
19 Gypsy Jazz
19 Polka
18 Cubano
18 Modal
17 Jump Blues
17 Louisiana Blues
15 Audiobook
15 Medieval
14 Chicago Blues
14 Merengue
13 Highlife
12 Beguine
12 Hillbilly
12 Soft Rock
11 Beat
11 Speech
10 Bossa Nova
10 Romani
9 Folk Rock
9 Interview
8 Piedmont Blues
8 Political
8 Quechua
8 Texas Blues
7 Avant-garde Jazz
7 Educational
7 Pasodoble
7 Salsa
6 East Coast Blues
6 Experimental
6 Harmonica Blues
6 Smooth Jazz
5 Andalusian Classical
5 Free Jazz
5 Gamelan
5 Hindustani
5 Sea Shanties
5 Sermon
5 Western Swing
4 Europop
4 Free Improvisation
4 Twelve-tone
4 Volksmusik
3 Abstract
3 Cajun
3 Funk
3 Hawaiian
3 Indian Classical
3 Musique Concrète
3 Steel Band
2 Acoustic
2 Afrobeat
2 Bachata
2 Baião
2 Batucada
2 Cape Jazz
2 Charanga
2 Contemporary R&B
2 Copla
2 Health-Fitness
2 Jazz-Funk
2 Laïkó
2 Mariachi
2 Post-Modern
2 Public Service Announcement
2 RnB/Swing
2 Zarzuela
1 Barbershop
1 Bollywood
1 Bossanova
1 Chinese Classical
1 Danzon
1 Electric Blues
1 Forró
1 Garage Rock
1 MPB
1 Maloya
1 Marcha Carnavalesca
1 Marimba
1 Modern Classical
1 Promotional
1 Public Broadcast
1 Rebetiko
1 Samba-Canção
1 Soca
1 Sound Poetry
1 Swamp Pop"""

In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="1950UKstyles.p")

In [ ]:
from collections import Counter
for item in Counter(refCounts).most_common():
    ref      = item[0]
    cnts     = item[1]
    discID   = arts.discogsUtils.getArtistID(ref)
    url      = arts.getArtistRef(ref)
    savename = arts.getArtistSavename(discID)
    if isFile(savename):
        continue
    try:
        arts.downloadArtistURL(url, savename)
    except:
        pass

In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="1980styles.p")

In [ ]:
ifile = "/Volumes/Music/Discog/collections/US-1960-Soundtrack-9.p"
from webUtils import getWebData, getHTML
from searchUtils import findPatternExt
verydebug=False
fdata  = getFile(ifile)
bsdata = getHTML(fdata)

albumsDir = disc.getAlbumsDir()

albs = albums(disc)


if True:
    collectionsDir  = col.getCollectionsDir()
    collectionFiles = findPatternExt(collectionsDir, pattern=".", ext=".p")
    print("Found {0} downloaded collection files".format(len(collectionFiles)))
    
    artsDowns = {}
    albsDowns = {}
    
    for i,ifile in enumerate(reversed(collectionFiles)):
        print("\n\n\n")
        print("Parsing {0}/{1}: {2}".format(i,len(collectionFiles), ifile))
        
        bsdata = getHTML(ifile)
        artistDB  = {}

        h4s = bsdata.findAll("h4")
        if verydebug:
            print("Found {0} h4 tags".format(len(h4s)))
        
        for ih4,h4 in enumerate(h4s):
            spans = h4.findAll("span")
            
            #### Get Artist
            ref   = None
            if len(spans) == 0:
                ref = h4.find("a")
            else:
                ref = spans[0].find("a")
                
            if ref is None:
                continue
                
            if artsDowns.get(ref) is True:
                continue
                
            try:
                artisthref = ref.attrs.get('href')
                artist     = ref.text.strip()
            except:
                print("Could not get artist/href from {0}".format(ref))
                continue
                
            
            #### Get Album
        
            try:
                ref        = h4.find("a", {"class": "search_result_title"})
                albumhref  = ref.attrs.get('href')
                album      = ref.text.strip()
            except:
                print("Could not get album/href from {0}".format(ref))
                continue

            
            #print("Artist: {0} ---> {1}".format(artist,artisthref))
            #print("Album:  {0} ---> {1}".format(album,albumhref))

            artistID  = arts.discogsUtils.getArtistID(artisthref)
            modValue  = arts.discogsUtils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
            
            
            ##### Download Artist Data
            url      = arts.getArtistRef(artisthref)
            savename = arts.getArtistSavename(artistID)
            if savename is not None:
                if not isFile(savename):
                    try:
                        arts.downloadArtistURL(url, savename)
                    except:
                        artsDowns[url] = True
                        continue
                #print('\t',url)
                #print('\t',savename)
                
        
            if modValue is None:
                artistModDir = setDir(albumsDir, "NAN")
                artistIDDir = setDir(artistModDir, "NAN")
            else:
                artistModDir = setDir(albumsDir, str(modValue))
                artistIDDir = setDir(artistModDir, artistID)


            ##### Download Album Data
            try:
                mediaID = int(getBasename(albumhref))
            except:
                raise ValueError("Not sure about {0}".format(h4))

            #print('\tAlbumDir:',artistIDDir)
            #print('\tMediaID: ',mediaID)

            if not isDir(artistIDDir):
                mkDir(artistIDDir)
            savename = setFile(artistIDDir, "{0}.p".format(mediaID))
            #print('\tSavename:',savename)
            if isFile(savename):
                continue
     
            #print('\t',savename)
            #print("")
            #continue

            baseURL = disc.discogURL
            url     = urllib.parse.urljoin(baseURL, quote(albumhref))
            
            
            if albsDowns.get(url) is True:
                continue
                
            try:
                albs.downloadAlbumURL(url, savename)
            except:
                albsDowns[url] = True
                continue

In [ ]:
#col.createCollectionsData()
#col.downloadCollections()
#col.parseCollections()
#col.mergeCollections(debug=True, force=True)
#col.createCollectionDBs(debug=True)

In [ ]:
disc.moveAlbumFilesToNewModValue(newModValue=100, oldModValue=500)

# Artist Section

In [ ]:
db = disc.getArtistNameToIDsData()

In [ ]:
#arts = artists(disc)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

# Albums Metadata

In [ ]:
artistNameToID   = disc.getArtistNameToIDData()
artistNameToIDs  = disc.getArtistNameToIDsData()
artistIDToName   = disc.getArtistIDToNameData()        
artistRefToID    = disc.getArtistRefToIDData()
artistIDToRef    = disc.getArtistIDToRefData()
artistRefToName  = disc.getArtistRefToNameData()
artistNameToRef  = disc.getArtistNameToRefData()
artistNameToRefs = disc.getArtistNameToRefsData()

In [ ]:
albumRefToName  = disc.getAlbumRefToNameData()
albumNameToRef  = disc.getAlbumNameToRefData()

artistIDCoreAlbumIDs   = disc.getArtistIDCoreAlbumNames()
artistIDAlbumIDs       = disc.getArtistIDAlbumNames()



In [ ]:
from collections import Counter
core = ["Albums"]
albumCntr = Counter()


print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
    "#","File","Core IDs-IDs","IDs-IDs","Name-IDs","Name-Refs","ID-Names","ID-Refs","Ref-IDs","Ref-Names"))


for i,dbfile in enumerate(dbfiles):
    db = getFile(dbfile)
    for artistID,artistData in db.items():
        artist      = artistData['Artist']
        artistName  = dutils.getArtistName(artist)
        artistRef   = artistData['URL']
        #print("{0: <20}{1: <50}{2}".format(artistID,artist,artistRef))
        
    print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
        i,dbfile,
        len(artistIDCoreAlbumIDs),len(artistIDAlbumIDs),
        len(albumNameToID),len(albumNameToRef),len(albumIDToName),
        len(albumIDToRef),len(albumRefToID),len(albumRefToName)))



In [ ]:
ifile="/Volumes/Music/Discog/artists/0/9900.p"
fdata = getHTML(getFile(ifile, version=2))
an = artistName(fdata)
an.getArtist()
#name = artistName(fdata)
#name


In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="soundtracks.p")

# Albums

In [ ]:
albs = albums(disc)
albs.parseAlbumFiles()

In [ ]:
getFile('/Volumes/Music/Discog/albums-db/NAN-DB.p')

In [ ]:
import random
albs = albums(disc)
nErrs = False
while nErrs is False:
    nErrs = True
    albs.downloadAlbumDataData(int(random.uniform(0,disc.getMaxModVal()))) #, mediaTypes=None)        
    if False:
        nErrs = False
        print("Sleeping for 60 seconds...")
        sleep(60)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
art = artist()
arts = artists(disc)

toget=getFile(ifile="toget3.p")
done = {}
i = 0
errs = {}
n = len(toget)
for artistID,artistRef in toget.items():
    i += 1
    ifile = arts.getArtistSavename(artistID)
    print(i,'/',n,'\t',ifile)
    info  = art.getData(ifile)
    pages = info.pages
    
    if pages.redo is True:
        url      = arts.getArtistURL(artistRef, 1)
        savename = arts.getArtistSavename(artistID, 1)
        try:
            arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)            
        except:
            errs[artistID] = artistRef
            print("\tErr with {0}".format(artistRef))

In [ ]:
import sqlite3
db = sqlite3.connect("test.db")
cur = db.cursor()
cur.execute("create table people (first varchar, last varchar, age integer)")
cur.execute('insert into people values ("Steve", "Cassidy", 21)')
cur.execute('insert into people values ("Mary", "Mullins", 23)')
db.commit()